In [9]:
from pathlib import Path
import os, sys
this_directory = Path(os.curdir).resolve().absolute()
project_directory = this_directory.parent
if str(project_directory) not in sys.path:
    sys.path.append(str(project_directory))
this_directory, project_directory

(PosixPath('/home/yecm/public/Multitask-Recommendation-Library/软件测试'),
 PosixPath('/home/yecm/public/Multitask-Recommendation-Library'))

In [10]:
import models
# categorical_field_dims=[3,2]
categorical_field_dims=[]
numerical_num=100
task_num=2

expert_num = 10
embed_dim=8
# bottom_mlp_dims=[3,4,5]
bottom_mlp_dims=[10]*100
tower_mlp_dims=[2,3,4] 
dropout=0.1
# layers_num = 10



ms = [models.get_model(name, categorical_field_dims, numerical_num, task_num, 
                       expert_num, embed_dim, 
                       bottom_mlp_dims, tower_mlp_dims, dropout,
                       ) for name in models.model_names]

Model: Shared-Bottom
Model: SingleTask
Model: OMoE
Model: MMoE
Model: PLE
Model: AITM
Model: MetaHeac


In [11]:
[len(list(m.shared_parameters())) for m in ms[:-1]], [len(list(m.specific_parameters())) for m in ms[:-1]]

([403, 6, 4005, 4003, 2203, 9], [28, 828, 28, 32, 4428, 830])

In [12]:
# https://blog.csdn.net/qq_33952811/article/details/124276599
# 参数量和计算量 = 空间复杂度、时间复杂度 = 显存大小、训练时间

# from torchstat import stat
from torchsummary import summary
from models import MultitaskWrapper
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
m = MultitaskWrapper(ms[4]).to(device)
# stat(m, (1, 4, 1))
summary(m, (numerical_num, ))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Embedding-1                 [-1, 0, 8]               0
    EmbeddingLayer-2                 [-1, 0, 8]               0
            Linear-3                    [-1, 8]             808
            Linear-4                   [-1, 10]              90
       BatchNorm1d-5                   [-1, 10]              20
              ReLU-6                   [-1, 10]               0
           Dropout-7                   [-1, 10]               0
MultiLayerPerceptron-8                   [-1, 10]               0
            Linear-9                   [-1, 10]              90
      BatchNorm1d-10                   [-1, 10]              20
             ReLU-11                   [-1, 10]               0
          Dropout-12                   [-1, 10]               0
MultiLayerPerceptron-13                   [-1, 10]               0
           Linear-14              

In [15]:
batch_size = 4
X = torch.rand((batch_size, numerical_num)).to(device)
# 默认分类任务，输出是一个数， 不过这里我们有batch个数
m(X)

[tensor([0.5736, 0.4327, 0.5736, 0.4600], device='cuda:0',
        grad_fn=<SigmoidBackward0>),
 tensor([0.4564, 0.4087, 0.4048, 0.3834], device='cuda:0',
        grad_fn=<SigmoidBackward0>)]

In [16]:
from tensorboardX import SummaryWriter
tensorboard = SummaryWriter(project_directory/'tensorboard_log/test')

In [17]:
tensorboard.add_graph(m, X)

2023-05-01 11:30:05.859582: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/yecm/.conda/envs/torch/lib/python3.11/site-packages/torch/jit/_trace.py:1056: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  module._c._create_method_from_trace(


In [18]:
torch.compile(m)

RuntimeError: Python 3.11+ not yet supported for torch.compile